In [ ]:
from glob import glob
from os.path import basename, splitext

def collect_scenarios(target_path, source_folder):
    tables = []
    for path in glob(join(source_folder, '*.csv')):
        table = pd.read_csv(path)
        table['ScenarioName'] = basename(splitext(path)[0])
        columns = list(table.columns)
        columns.insert(0, columns.pop(columns.index('ScenarioName')))
        tables.append(table[columns])
    scenarios = pd.concat(tables)
    scenarios.to_csv(target_path, index=False)
    return scenarios
    
scenarios = collect_scenarios(join(output_folder, 'details.csv'), details_folder)
summaries = collect_scenarios(join(output_folder, 'summaries.csv'), summaries_folder)
scenarios.head

In [ ]:
report_year = summaries.columns[-1]
report_year

In [ ]:
t = summaries[['Unnamed: 0', report_year]]
t = t.set_index('Unnamed: 0')
t = t.squeeze()
t = t[t != 0]
t

In [ ]:
population_keys = [_ for _ in t.keys() if _.startswith('1.Population')]
population_keys

In [ ]:
technology_codes = [_.replace('1.Population_', '') for _ in population_keys]
technology_codes

In [ ]:
technology_name_by_code = {
    'Grid': 'Grid Extension',
    'SA_PV': 'Stand Alone - Photovoltaic',
    'MG_PV': 'Mini Grid - Photovoltaic',
    'MG_Wind': 'Mini Grid - Wind',
}
technology_names = [technology_name_by_code[_] for _ in technology_codes]
technology_names

In [ ]:
import pandas as pd
summary_table = pd.DataFrame()
summary_table['Technology'] = technology_names

In [ ]:
t

In [ ]:
population_index = '1.Population'
summary_table['Population Connected'] = t[t.index.str.contains(population_index)].reset_index(drop=True)
summary_table

In [ ]:
investment_index = '4.Investment'
summary_table['Investment Required'] = t[t.index.str.contains(investment_index)].reset_index(drop=True)
summary_table


In [ ]:
added_capacity_index = '3.Capacity'
summary_table['Added Capacity'] = t[t.index.str.contains(added_capacity_index)].reset_index(drop=True)

with open(join(output_folder, "overview-table.geojson"), 'w') as f:
    summary_table.to_csv(f)
summary_table


In [ ]:
import geopandas
import matplotlib.pyplot as plt
import json

world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

fig, ax = plt.subplots(1, 1, figsize=(9, 7))
ax.set_aspect('equal')
ax.set_yticklabels([])
ax.set_xticklabels([])

ax.axis("off")

cell_positions = scenarios[['X_deg', 'Y_deg', 'Pop', 'GridCellArea', 'Country']]

gdf = geopandas.GeoDataFrame(
    cell_positions, geometry=geopandas.points_from_xy(scenarios.X_deg, scenarios.Y_deg))

collection = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [row['X_deg'], row['Y_deg']]
      },
      "properties": {
        "Population": row['Pop'],
        'GridCellArea': row['GridCellArea']
      }
    } for index, row in gdf.iterrows()
  ]
}

#ax = world[world.name == 'Djibouti'].plot(
#    color='#F0F0F0', edgecolor='#333333', ax=ax)

# We can now plot our ``GeoDataFrame``.
#gdf.plot(markersize='GridCellArea', cmap='OrRd', ax=ax)

with open(join(output_folder, "overview-map.csv"), 'w') as f:
    json.dump(collection ,f)
#gdf.to_file(, driver='GeoJSON')

#plt.tight_layout()
#plt.show()

#plt.savefig(join(output_folder, 'figure1.png'))

# collection